# Importing Libraries

In [1]:
import nltk

# nltk.download()

In [2]:
# Import libraries

import pandas as pd
import re 

import nltk 
from nltk.tokenize import RegexpTokenizer
from collections import Counter 
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost 
from sklearn.metrics  import classification_report
from sklearn import metrics
import time



C:\Users\User\anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


# Combining multiple excel files into one dataframe


In [3]:
import os
import pandas as pd
cwd = os.path.abspath('CSV Data for Classifier') 
files = os.listdir(cwd) 

In [4]:
cwd

'C:\\Users\\User\\Desktop\\FYP\\FSX\\News Classifier\\CSV Data for Classifier'

In [5]:
files

['avalanche_straitstimes.csv',
 'contamination_straitstimes.csv',
 'cyberattack_straitstimes.csv',
 'cyclone_straitstimes.csv',
 'dengue_straitstimes.csv',
 'drought_straitstimes.csv',
 'earthquake_straitstimes.csv',
 'ebola_straitstimes.csv',
 'economic crisis_straitstimes.csv',
 'floods_straitstimes.csv',
 'heat stress_straitstimes.csv',
 'Influenza_straitstimes.csv',
 'limnic eruption_straitstimes.csv',
 'nuclear_straitstimes.csv',
 'oil spills_straitstimes.csv',
 'pandemic_straitstimes.csv',
 'SarS_straitstimes.csv',
 'sinkhole_straitstimes.csv',
 'terror_straitstimes.csv',
 'tradedispute_straitstimes.csv',
 'tsunami_straitstimes.csv',
 'Unsafe_straitstimes.csv',
 'volcanic eruption_straitstimes.csv',
 'war_straitstimes.csv']

In [6]:
df = pd.DataFrame()
for file in files:
     if file.endswith('.csv'):
         df = df.append(pd.read_csv(cwd+"/"+file), ignore_index=True) 
df.head()

<ipython-input-6-c499f365edd9>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.read_csv(cwd+"/"+file), ignore_index=True)
<ipython-input-6-c499f365edd9>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.read_csv(cwd+"/"+file), ignore_index=True)
<ipython-input-6-c499f365edd9>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.read_csv(cwd+"/"+file), ignore_index=True)
<ipython-input-6-c499f365edd9>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.read_csv(cwd+"/"+file), ignore_index=True)
<ipython-input-6-c499f365edd9>:4: FutureWarning: The frame.append method is depr

,Unnamed: 0,date,location,news title,news source(url),content summary,keywords,class_name,new_class_name
0,0,Feb-22,MUMBAI (REUTERS),Himalayan avalanche kills seven Indian soldier...,https://www.straitstimes.com/asia/himalayan-av...,MUMBAI (REUTERS) - A Himalayan avalanche kille...,"kills, avalanche, defence, kameng, china, sold...",natural calamities,geophysical event
1,1,Feb-22,ZURICH (REUTERS),Eight killed in two days after third deadly av...,https://www.straitstimes.com/world/europe/eigh...,ZURICH (REUTERS) - One person was killed and f...,"person, avalanche, tyrol, killed, days, skiers...",natural calamities,geophysical event
2,2,Feb-22,VIENNA (REUTERS),Avalanche in Austria near Swiss border kills five,https://www.straitstimes.com/world/europe/aval...,VIENNA (REUTERS) - An avalanche in an area of ...,"kills, avalanche, person, services, supervisor...",natural calamities,geophysical event
3,3,Feb-22,NEW DELHI (REUTERS),Himalayan avalanche traps Indian Army patrol t...,https://www.straitstimes.com/asia/south-asia/h...,NEW DELHI (REUTERS) - A Himalayan avalanche tr...,"arunachal, avalanche, army, defence, team, chi...",natural calamities,geophysical event
4,4,Feb-21,SALT LAKE CITY (NYTIMES),"4 skiers killed in avalanche in Utah, official...",https://www.straitstimes.com/world/united-stat...,SALT LAKE CITY (NYTIMES) - Four back-country s...,"avalanche, states, killed, skiers, saidthe, sa...",natural calamities,geophysical event


In [7]:
df.shape

(13747, 9)

In [8]:
df['new_class_name'].value_counts()

Acute climatological event (droughts)       1743
idiosyncratic                               1320
Acute climatological event (flood)          1256
geophysical event                           1174
cyberattack                                 1117
Terrorism                                   1050
Military Conflicts                          1022
Economic Crisis                             1018
pandemic                                    1000
Acute climatological event (cyclone)         887
Trade Dispute                                809
Man-Made Disaster                            801
Acute climatological event (heat stress)     550
Name: new_class_name, dtype: int64

# Data Exploration

In [9]:
category = list(df['new_class_name'].unique())
category

['geophysical event',
 'idiosyncratic',
 'cyberattack',
 'Acute climatological event (cyclone)',
 'pandemic',
 'Acute climatological event (droughts)',
 'Economic Crisis',
 'Acute climatological event (flood)',
 'Acute climatological event (heat stress)',
 'Man-Made Disaster',
 'Terrorism',
 'Trade Dispute',
 'Military Conflicts']

In [36]:
mymap = {
    'geophysical event' : 0,
    'idiosyncratic' : 1,
    'cyberattack' : 2,
    'Acute climatological event (cyclone)' : 3,
    'pandemic':4,
    'Acute climatological event (droughts)':5,
    'Economic Crisis':6,
    'Acute climatological event (flood)':7,
    'Acute climatological event (heat stress)':8,
    'Man-Made Disaster':9,
    'Terrorism':10,
    'Trade Dispute':11,
    'Military Conflicts':12,
}

df["new_class_name_index"] = df['new_class_name'].apply(lambda x: mymap.get(x))
df.tail(10)

,Unnamed: 0,date,location,news title,news source(url),content summary,keywords,class_name,new_class_name,Preprocessed_Text,Preprocessed_Text2,new_class_name_index
13737,1012,Aug-20,SEOUL (BLOOMBERG),"Japan, South Korea tensions renewed at World W...",https://www.straitstimes.com/asia/east-asia/ja...,SEOUL (BLOOMBERG) - Tense ties between major U...,"seoul, south, war, ii, allies, abe, world, cer...",Conflict/Terrorist activities,Military Conflicts,"[seoul, bloomberg, tense, tie, major, ally, ja...",seoul bloomberg tense tie major ally japan sou...,12
13738,1013,Aug-20,SINGAPORE,Private ceremony at Kranji Memorial to mark 75...,https://www.straitstimes.com/singapore/private...,"Emperor Hirohito, in a historical speech broad...","war, soldiers, singapore, known, million, 75th...",Conflict/Terrorist activities,Military Conflicts,"[emperor, hirohito, historical, speech, broadc...",emperor hirohito historical speech broadcast r...,12
13739,1014,Nov-20,After Mr Joe Biden was announced as the winner...,Actor Mark Hamill tweets Star Wars-themed post...,https://www.straitstimes.com/life/entertainmen...,After Mr Joe Biden was announced as the winner...,"trump, president, hamill, star, wars, warsthem...",Conflict/Terrorist activities,Military Conflicts,"[joe, biden, announced, winner, united, state,...",joe biden announced winner united state presid...,12
13740,1015,Jan-21,SEOUL • A South Korean court yesterday ordered...,Japan must pay ex-sex slaves: Seoul court,https://www.straitstimes.com/asia/east-asia/ja...,SEOUL • A South Korean court yesterday ordered...,"seoul, pay, south, victims, slaves, japan, rul...",Conflict/Terrorist activities,Military Conflicts,"[seoul, south, korean, court, yesterday, order...",seoul south korean court yesterday ordered jap...,12
13741,1016,Mar-21,"MAE SAM LAEP, THAILAND (AFP)",Dozens of Myanmar refugees make 'voluntary ret...,https://www.straitstimes.com/asia/se-asia/doze...,"MAE SAM LAEP, THAILAND (AFP) - Some 50 Myanmar...","dozens, laep, karen, mae, voluntary, thai, mya...",Conflict/Terrorist activities,Military Conflicts,"[mae, sam, laep, thailand, afp, myanmar, refug...",mae sam laep thailand afp myanmar refugee fled...,12
13742,1017,Mar-21,BANGKOK,Myanmar's ethnic armed groups weigh the costs ...,https://www.straitstimes.com/asia/se-asia/myan...,BANGKOK - In the first few weeks after the Feb...,"thinon, war, looms, weigh, protesters, stop, m...",Conflict/Terrorist activities,Military Conflicts,"[bangkok, first, week, feb, coup, myanmar, hop...",bangkok first week feb coup myanmar hopeful pr...,12
13743,1018,Sep-20,BANGKOK (NYTIMES),World War II submarine found in murky waters o...,https://www.straitstimes.com/asia/se-asia/worl...,BANGKOK (NYTIMES) - In the murky waters of the...,"grenadier, war, ii, waters, robert, malacca, w...",Conflict/Terrorist activities,Military Conflicts,"[bangkok, nytimes, murky, water, strait, malac...",bangkok nytimes murky water strait malacca km ...,12
13744,1019,Nov-20,KUALA LUMPUR • Malaysia plans to boost its Covid,Covid-19 war chest being beefed up,https://www.straitstimes.com/asia/se-asia/covi...,KUALA LUMPUR • Malaysia plans to boost its Cov...,"war, pandemic, billion, covid19, vaccine, beef...",Conflict/Terrorist activities,Military Conflicts,"[kuala, lumpur, malaysia, plan, boost, covid, ...",kuala lumpur malaysia plan boost covid war che...,12
13745,1020,Mar-21,"MAE SAM LAEP, THAILAND (REUTERS)","Some Myanmar wounded cross into Thailand, othe...",https://www.straitstimes.com/asia/se-asia/thai...,"MAE SAM LAEP, THAILAND (REUTERS) - Seven peopl...","laep, cross, asked, border, mae, wounded, myan...",Conflict/Terrorist activities,Military Conflicts,"[mae, sam, laep, thailand, reuters, seven, peo...",mae sam laep thailand reuters seven people fle...,12
13746,1021,May-20,LOS ANGELES (REUTERS),Disney announces new Star Wars theatrical film...,https://www.straitstimes.com/lifestyle/enterta...,"LOS ANGELES (REUTERS) - Oscar-winning ""Jojo Ra...","disney, series, star,

# Text Preprocessing

- One way to improve on this could be to remove "said" or irrelevant words to clean the text more explicity

In [12]:
def preprocess(text):
    
    """
    Function: split text into words and return the root form of the words
    Args:
      text(str): the article
    Return:
      lem(list of str): a list of the root form of the article words
    """
        
    # Normalize text
    text = re.sub(r"[^a-zA-Z]", " ", str(text).lower())
    
    # Tokenize text
    token = word_tokenize(text)
    
    # Remove stop words
    stop = stopwords.words("english")
    new_stop_words_list = ['said', 'us', 'also', 'mr']
    stop.extend(new_stop_words_list)
    words = [t for t in token if t not in stop]
    
    # Lemmatization
    lem = [WordNetLemmatizer().lemmatize(w) for w in words]
    
    return lem

In [13]:
df["Preprocessed_Text"] = df['content summary'].apply(lambda x: preprocess(x))
df.head(10)

,Unnamed: 0,date,location,news title,news source(url),content summary,keywords,class_name,new_class_name,Preprocessed_Text
0,0,Feb-22,MUMBAI (REUTERS),Himalayan avalanche kills seven Indian soldier...,https://www.straitstimes.com/asia/himalayan-av...,MUMBAI (REUTERS) - A Himalayan avalanche kille...,"kills, avalanche, defence, kameng, china, sold...",natural calamities,geophysical event,"[mumbai, reuters, himalayan, avalanche, killed..."
1,1,Feb-22,ZURICH (REUTERS),Eight killed in two days after third deadly av...,https://www.straitstimes.com/world/europe/eigh...,ZURICH (REUTERS) - One person was killed and f...,"person, avalanche, tyrol, killed, days, skiers...",natural calamities,geophysical event,"[zurich, reuters, one, person, killed, four, o..."
2,2,Feb-22,VIENNA (REUTERS),Avalanche in Austria near Swiss border kills five,https://www.straitstimes.com/world/europe/aval...,VIENNA (REUTERS) - An avalanche in an area of ...,"kills, avalanche, person, services, supervisor...",natural calamities,geophysical event,"[vienna, reuters, avalanche, area, austria, bo..."
3,3,Feb-22,NEW DELHI (REUTERS),Himalayan avalanche traps Indian Army patrol t...,https://www.straitstimes.com/asia/south-asia/h...,NEW DELHI (REUTERS) - A Himalayan avalanche tr...,"arunachal, avalanche, army, defence, team, chi...",natural calamities,geophysical event,"[new, delhi, reuters, himalayan, avalanche, tr..."
4,4,Feb-21,SALT LAKE CITY (NYTIMES),"4 skiers killed in avalanche in Utah, official...",https://www.straitstimes.com/world/united-stat...,SALT LAKE CITY (NYTIMES) - Four back-country s...,"avalanche, states, killed, skiers, saidthe, sa...",natural calamities,geophysical event,"[salt, lake, city, nytimes, four, back, countr..."
5,5,Apr-21,NEW DELHI (XINHUA),8 killed after glacial burst triggers avalanch...,https://www.straitstimes.com/asia/south-asia/8...,"According to officials, 384 others, mostly bel...","avalanche, officials, bro, glacial, uttarakhan...",natural calamities,geophysical event,"[according, official, others, mostly, belongin..."
6,6,Jan-21,VIENNA (REUTERS),Dogs' barking prompts owners' rescue from Swis...,https://www.straitstimes.com/world/europe/dogs...,VIENNA (REUTERS) - Two people caught in an ava...,"dogs, avalanche, buried, snowshoers, group, wi...",natural calamities,geophysical event,"[vienna, reuters, two, people, caught, avalanc..."
7,7,Jan-21,FRANCE (AFP),'Miracle' escape for man trapped in Alps avala...,https://www.straitstimes.com/world/europe/mira...,FRANCE (AFP) - A man out walking with his fami...,"minutes, avalanche, local, escape, snow, rescu...",natural calamities,geophysical event,"[france, afp, man, walking, family, french, al..."
8,8,Jan-20,"MUZAFFARABAD, PAKISTAN (REUTERS)",Girl buried under Pakistan avalanche for 18 ho...,https://www.straitstimes.com/asia/south-asia/g...,"MUZAFFARABAD, PAKISTAN (REUTERS) - A 12-year-o...","buried, avalanche, waited, pakistan, village, ...",natural calamities,geophysical event,"[muzaffarabad, pakistan, reuters, year, old, g..."
9,9,Dec-19,VIENNA (REUTERS),"Rescuers comb Austrian, Swiss avalanches in ca...",https://www.straitstimes.com/world/europe/resc...,VIENNA (REUTERS) - Rescuers hunted for possibl...,"avalanche, large, snow, warning, swiss, victim...",natural calamities,geophysical event,"[vienna, reuters, rescuer, hunted, possible, v..."


# Text Exploration

In [14]:
def find_common_words(df, category):
        
    """
    Function: find the most frequent words in the category and return the them
    Args:
      df(dataframe): the dataframe of articles
      category(str): the category name
    Return:
      the most frequant words in the category
    """
        
    # Create dataframes for the category
    cat_df = df[df["new_class_name"]==category]
    
    # Initialize words list for the category
    words = [word for tokens in cat_df["Preprocessed_Text"] for word in tokens]
    
    # Count words frequency
    words_counter = Counter(words)
 
    return words_counter.most_common(10)

In [15]:
print("Most common words in each category")
for c in category:
    print(c, " News")
    print(find_common_words(df, c))
    print()

Most common words in each category
geophysical event  News
[('people', 677), ('volcano', 580), ('eruption', 448), ('tsunami', 397), ('year', 391), ('earthquake', 386), ('island', 377), ('quake', 353), ('new', 336), ('one', 325)]

idiosyncratic  News
[('year', 579), ('singapore', 450), ('food', 376), ('one', 368), ('safety', 356), ('people', 346), ('last', 328), ('new', 311), ('health', 299), ('unsafe', 283)]

cyberattack  News
[('cyber', 2317), ('attack', 1604), ('security', 1483), ('singapore', 771), ('data', 567), ('system', 527), ('government', 471), ('agency', 419), ('hacker', 386), ('year', 370)]

Acute climatological event (cyclone)  News
[('cyclone', 747), ('people', 619), ('storm', 603), ('climate', 446), ('year', 384), ('state', 365), ('island', 329), ('wind', 303), ('hit', 295), ('tropical', 293)]

pandemic  News
[('dengue', 955), ('year', 780), ('case', 767), ('singapore', 583), ('health', 523), ('covid', 514), ('virus', 484), ('people', 477), ('pandemic', 445), ('last', 362

In [16]:
df['Preprocessed_Text2'] = df['Preprocessed_Text'].apply(' '.join)
df.head()

,Unnamed: 0,date,location,news title,news source(url),content summary,keywords,class_name,new_class_name,Preprocessed_Text,Preprocessed_Text2
0,0,Feb-22,MUMBAI (REUTERS),Himalayan avalanche kills seven Indian soldier...,https://www.straitstimes.com/asia/himalayan-av...,MUMBAI (REUTERS) - A Himalayan avalanche kille...,"kills, avalanche, defence, kameng, china, sold...",natural calamities,geophysical event,"[mumbai, reuters, himalayan, avalanche, killed...",mumbai reuters himalayan avalanche killed seve...
1,1,Feb-22,ZURICH (REUTERS),Eight killed in two days after third deadly av...,https://www.straitstimes.com/world/europe/eigh...,ZURICH (REUTERS) - One person was killed and f...,"person, avalanche, tyrol, killed, days, skiers...",natural calamities,geophysical event,"[zurich, reuters, one, person, killed, four, o...",zurich reuters one person killed four others i...
2,2,Feb-22,VIENNA (REUTERS),Avalanche in Austria near Swiss border kills five,https://www.straitstimes.com/world/europe/aval...,VIENNA (REUTERS) - An avalanche in an area of ...,"kills, avalanche, person, services, supervisor...",natural calamities,geophysical event,"[vienna, reuters, avalanche, area, austria, bo...",vienna reuters avalanche area austria borderin...
3,3,Feb-22,NEW DELHI (REUTERS),Himalayan avalanche traps Indian Army patrol t...,https://www.straitstimes.com/asia/south-asia/h...,NEW DELHI (REUTERS) - A Himalayan avalanche tr...,"arunachal, avalanche, army, defence, team, chi...",natural calamities,geophysical event,"[new, delhi, reuters, himalayan, avalanche, tr...",new delhi reuters himalayan avalanche trapped ...
4,4,Feb-21,SALT LAKE CITY (NYTIMES),"4 skiers killed in avalanche in Utah, official...",https://www.straitstimes.com/world/united-stat...,SALT LAKE CITY (NYTIMES) - Four back-country s...,"avalanche, states, killed, skiers, saidthe, sa...",natural calamities,geophysical event,"[salt, lake, city, nytimes, four, back, countr...",salt lake city nytimes four back country skier...


In [37]:
X = df['Preprocessed_Text2']
y = df['new_class_name_index']

In [38]:
y

0         0
1         0
2         0
3         0
4         0
         ..
13742    12
13743    12
13744    12
13745    12
13746    12
Name: new_class_name_index, Length: 13747, dtype: int64

# Feature Extraction

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [40]:
y_test

11885     1
8673      9
11074    11
12136     1
3562      5
         ..
11511     1
7808      8
4069      5
10784    11
11437     0
Name: new_class_name_index, Length: 2750, dtype: int64

In [26]:
tf_vec = TfidfVectorizer()
x_train_features = tf_vec.fit_transform(X_train)
x_test_features = tf_vec.fit_transform(X_test)

<2750x22 sparse matrix of type '<class 'numpy.float64'>'
	with 6516 stored elements in Compressed Sparse Row format>

In [41]:
from tpot import TPOTClassifier

pipeline_optimizer = TPOTClassifier(generations=5, population_size=20, cv=5,
                                    random_state=42, verbosity=2, config_dict = 'TPOT sparse')
pipeline_optimizer.fit(x_train_features, y_train)
print(pipeline_optimizer.score(x_test_features, y_test))
pipeline_optimizer.export('tpot_exported_pipeline.py')

Optimization Progress:   0%|          | 0/120 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.1273074124602092

Generation 2 - Current best internal CV score: 0.1273074124602092

Generation 3 - Current best internal CV score: 0.1273074124602092

Generation 4 - Current best internal CV score: 0.1273074124602092

Generation 5 - Current best internal CV score: 0.1273074124602092

Best pipeline: MultinomialNB(input_matrix, alpha=10.0, fit_prior=False)


ValueError: X has 18947 features, but MultinomialNB is expecting 34651 features as input.